In [1]:
from glob import glob
import json
from tqdm import tqdm
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import json

In [2]:
import json
import jsonlines

data_list = []

with jsonlines.open('carigold-pair-result.jsonl') as reader:
    for line in reader:
        # Each line is a dictionary representing a JSON object
        # You can work with the data as a Python dictionary
        data_list.append(line)


In [3]:
len(data_list)

984668

In [23]:
def convert_keys_to_int(d):
    return [int(key) for key in d.keys()][0]

list_done = [convert_keys_to_int(d) for d in data_list]

In [25]:
len(list_done)

624714

In [26]:
data = []

with open('ada-002-carigold.jsonl') as fopen:
    for x in fopen:
        data.append(json.loads(x))

In [27]:
len(data)

984668

In [35]:
list_done_set = set(list_done)
carigold = [x for i, x in enumerate(data) if i not in list_done_set]

In [4]:
import math

math.comb(len(data),2)

484785042778

In [43]:
vectors, texts = [], []

for d in carigold:
    vectors.append(d['v'])
    texts.append(d['cleaned_text'])

In [44]:
data[0]['cleaned_text']

'Tuan TT ada lg stok x? boleh PM harga? nk PM tuan x lepas'

In [45]:
texts[0]

'aduh2.... nie memang betul marah nie...sabar tu separuh dari iman... aku tahu kau invest banyak kat sini.. so jadikan pengajaran... lain kali kalau nak invest tu ikut kemampuan.. yang kau hentam tarok...je.... gi main pakai egold lak....'

In [46]:
from scipy.spatial import KDTree
import numpy as np

In [47]:
concat = np.array(vectors)
concat.shape

(359954, 1536)

In [48]:
%%time

kd_tree = KDTree(concat, leafsize = 40)

CPU times: user 5.17 s, sys: 36.4 ms, total: 5.21 s
Wall time: 5.22 s


In [49]:
concat[0]

array([ 0.00702854, -0.02937107,  0.01440337, ..., -0.01658376,
        0.0297815 , -0.01759699])

In [50]:
texts[0]

'aduh2.... nie memang betul marah nie...sabar tu separuh dari iman... aku tahu kau invest banyak kat sini.. so jadikan pengajaran... lain kali kalau nak invest tu ikut kemampuan.. yang kau hentam tarok...je.... gi main pakai egold lak....'

In [52]:
def convert_int64(obj):
    if isinstance(obj, np.int64):
        return int(obj)
    raise TypeError

In [59]:
cont = int(list(data_list[-1].keys())[0])
cont

624713

In [61]:
data_list[-1]

{'624713': {'positive': [634318,
   533064,
   259359,
   336156,
   801994,
   651339,
   150099,
   338071,
   584673,
   258200,
   555821,
   5542,
   65615,
   675286,
   501269,
   810253,
   675313,
   541795,
   516608,
   855861,
   229319,
   434010,
   131984,
   218909,
   753652,
   157057,
   412028,
   800959,
   911904,
   702311,
   730675,
   753527,
   198339,
   676795,
   531796,
   801530,
   291904,
   135841,
   254619,
   225284,
   888767,
   397402,
   683044,
   417410,
   327248,
   324695,
   417000,
   857886,
   688549,
   811712,
   557115,
   728280,
   928572,
   308040,
   247679,
   177552,
   719514,
   774585,
   554525,
   258273,
   418440,
   284784,
   858286,
   667579,
   240506,
   962457,
   377207,
   948938,
   326397,
   745617,
   753520,
   3635,
   5409,
   633002,
   237144,
   187588,
   674538,
   31230,
   434522,
   494721,
   43621,
   872667,
   77190,
   417023,
   72142,
   333162,
   267279,
   6055,
   204932,
   323951,
 

In [60]:
carigold[0]

{'v': [0.007028536405414343,
  -0.029371073469519615,
  0.014403369277715683,
  -0.032038841396570206,
  -0.046147216111421585,
  0.0017106406157836318,
  -0.017096787691116333,
  -0.024471620097756386,
  -0.0178535096347332,
  -0.018969353288412094,
  -0.0003450941003393382,
  0.020983003079891205,
  -0.01191516499966383,
  0.01206907443702221,
  0.006368007976561785,
  0.0009346799342893064,
  0.02518986351788044,
  -0.004508267622441053,
  0.003950345329940319,
  -0.0025266818702220917,
  -0.012845035642385483,
  0.012376893311738968,
  0.012312764301896095,
  -0.008811322040855885,
  -0.014608582481741905,
  -0.007073426619172096,
  0.014813794754445553,
  -0.017135264351963997,
  -0.017661122605204582,
  -0.02017497830092907,
  0.001006023376248777,
  0.0014196553966030478,
  -0.011697126552462578,
  0.0045242998749017715,
  -0.023650769144296646,
  0.002985204104334116,
  0.002701433375477791,
  0.022291233763098717,
  0.025574637576937675,
  0.008561219088733196,
  0.01549356244

In [ ]:
dist_list = []
ind_list = []
indices = []

def compute_neighbors(args):
    idx, x = args
    dist, ind = kd_tree.query(x, k=len(concat))
    
    result = {
        'positive': [],
        'negative': []
    }
    
    result['positive'] = [x for x in ind[dist <= 0.45] if x != idx]
    result['negative'] = [x for x in ind[dist > 0.85]]
    
    return {str(cont+idx+1): result}

with ThreadPoolExecutor(max_workers=30) as executor:        
    futures = [executor.submit(compute_neighbors, x) for x in enumerate(concat)]
    
    for future in tqdm(futures, total=len(concat)):
        result = future.result()
        if result:
            with open('carigold-pair-result.jsonl', 'a') as f:
                json.dump(result,f, default = convert_int64)
                f.write('\n')

 51%|█████     | 182570/359954 [3:37:00<3:20:13, 14.77it/s] 

In [40]:
# with open('facebook-pair-mining.json') as json_file:
#     dat = json.load(json_file)

In [42]:
dat['50']

{'positive': [43644, 78217, 15770, 91392, 172623, 90454, 168396],
 'negative': [159503,
  145138,
  63754,
  181566,
  10828,
  167890,
  127536,
  29695,
  106669,
  39862,
  107284,
  29126,
  15123,
  131744,
  42373,
  2819,
  118357,
  69860,
  101545,
  20483,
  30543,
  9985]}

In [48]:
texts[168396]

'Saifudin abdullah'

In [44]:
texts[50]

'Mohd Fuad Abdullah Sani kah'